# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


In [67]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


In [68]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_train.head()

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [69]:
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_test.head()

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
id,,,,,,,,,,,,,,,,,,,,,
16115,D,76.89,material_7,material_5,6,6,16,2,7,17.427,...,18.241,11.536,14.690,19.622,11.082,16.323,15.633,16.153,15.640,648.974
16116,D,96.13,material_7,material_5,6,6,9,10,5,18.014,...,19.056,10.746,NaN,18.475,10.174,15.710,16.274,12.229,17.204,741.103
16117,D,124.40,material_7,material_5,6,6,10,15,0,18.374,...,20.689,10.240,16.035,18.224,11.005,15.925,16.257,16.338,17.653,NaN
16118,D,175.84,material_7,material_5,6,6,4,8,15,18.056,...,19.442,12.015,18.195,19.410,10.761,16.176,16.926,13.813,13.615,NaN
16119,D,118.70,material_7,material_5,6,6,15,8,5,16.513,...,19.007,12.615,15.775,18.778,9.683,NaN,17.066,14.690,18.798,NaN


In [70]:
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')
df_ans.head()

,failure
id,
16115,1
16116,0
16117,0
16118,1
16119,0


In [71]:
df_train['product_code'].unique()

array(['A', 'B', 'C', 'E'], dtype=object)

In [72]:
df_test['product_code'].unique()

array(['D'], dtype=object)

In [73]:
# 문제 1에서 failure와 연관성있는 결측치 변수를 추가합니다.
df_train['na_1'] = df_train['measurement_3'].isna()
df_train['na_2'] = df_train['measurement_5'].isna()
df_test['na_1'] = df_test['measurement_3'].isna()
df_test['na_2'] = df_test['measurement_5'].isna()

In [74]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

# 모델을 생성합니다
def create_imp():
    return  IterativeImputer(LinearRegression(),random_state = 123)

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
# Test에 Train에서 등장하지 않은 product_code가 있습니다.
# product_code 별로 만드는 것이라, test와 결합하여 모델을 만듭니다.
# product_code 각각에 대해 create_imp로 모델을 생성하고, 학습을 시킵니다.
s_imp = pd.concat([
    df_train[X_imp + ['product_code']],
    df_test[X_imp + ['product_code']]
]).groupby('product_code').apply(lambda x: create_imp().fit(x[X_imp]))

# 학습한 모델을 가지고, 결측치를 보완합니다.
df_train[X_imp] = df_train.groupby('product_code').apply(
    lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
)
df_test[X_imp] = df_test.groupby('product_code').apply(
    lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
)

In [75]:
cols = ['measurement_{}'.format(i) for i in range(10, 17)]
# Test에 Train에서 등장하지 않은 product_code가 있습니다.
# 병합하여 통계를 냅니다.
df_mean = pd.concat([
    df_train[cols + ['product_code']],
    df_test[cols + ['product_code']],
]).groupby('product_code')[cols].mean()
df_train[cols] = df_train.groupby('product_code')[cols].apply(
    lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index)
)
df_test[cols] = df_test.groupby('product_code')[cols].apply(
    lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index)
)

In [76]:
# loading의 결측치를 처리합니다.
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())

In [89]:
# 결측여부를 체크합니다.
df_train.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0)

In [78]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

In [79]:
# baseline 모델을 만듭니다. PCA(n_components=7)을 넣습니다.
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
m_cols = ['measurement_{}'.format(i) for i in range(0, 18)]
# ColumnTransformer를 통한 PCA를 포함한 전처리기를 구성합니다.
ct_pca = ColumnTransformer([
    ('std', StandardScaler(), ['loading']), 
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=7)), m_cols), 
    ('pt', 'passthrough', ['na_1', 'na_2'])
])
clf_lr = make_pipeline(ct_pca, LogisticRegression(solver='lbfgs'))
X_lr = m_cols + ['loading', 'na_1', 'na_2']
scores_ = cross_val_score(clf_lr, df_train[X_lr], df_train['failure'], scoring='roc_auc', cv=cv)
scores_, np.mean(scores_)

(array([0.58193084, 0.5891706 , 0.57115559, 0.61209017, 0.58374776]),
 0.5876189917347917)

In [80]:
clf_lr.fit(df_train[X_lr], df_train['failure'])
pd.DataFrame(
    clf_lr.predict_proba(df_test[X_lr])[:, 1],
    columns=['failure'],
    index=df_test.index
).to_csv('answer6.csv') # , index=None 인덱스는 제외하고자 할때

In [81]:
roc_auc_score(df_ans['failure'], clf_lr.predict_proba(df_test[X_lr])[:, 1])

0.5907111061151079

In [82]:
# 전진선택법으로 도출한 속성으로 모델을 만들어 봅니다.
ct_2 = ColumnTransformer([
    ('std', StandardScaler(), ['loading','measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']), 
    ('pt', 'passthrough', ['na_1'])
])
clf_lr_2 = make_pipeline(ct_2, LogisticRegression(solver='lbfgs'))
X_lr_2 = ['loading','measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']
scores_ = cross_val_score(clf_lr_2, df_train[X_lr_2], df_train['failure'], scoring='roc_auc', cv=cv)
scores_, np.mean(scores_)

(array([0.58840017, 0.5909264 , 0.57725538, 0.61231432, 0.58375591]),
 0.590530435171204)

In [83]:
clf_lr_2.fit(df_train[X_lr_2], df_train['failure'])
roc_auc_score(df_ans['failure'], clf_lr_2.predict_proba(df_test[X_lr_2])[:, 1])

0.5883988309352517

In [86]:
# VotingClassifier를 만들어 봅니다.
# soft voting을 하여 확률 기반으로 결과를 도출합니다.
from sklearn.ensemble import VotingClassifier
clf_vt = VotingClassifier([
    ('lr_1', clf_lr),
    ('lr_2', clf_lr_2),
], voting='soft')
scores_ = cross_val_score(clf_vt, df_train, df_train['failure'], scoring='roc_auc', cv=cv)
scores_, np.mean(scores_)

(array([0.58614124, 0.59083755, 0.57488773, 0.61284115, 0.58439286]),
 0.5898201073026861)

In [88]:
clf_vt.fit(df_train[df_test.columns], df_train['failure'])
roc_auc_score(df_ans['failure'], clf_vt.predict_proba(df_test)[:, 1])

0.5901139838129497

In [91]:
# 실제 상황이었다면, 교차검증결과가 가장 좋은 모델을 제출했을 껍니다.
clf_lr_2.fit(df_train[X_lr_2], df_train['failure'])
pd.DataFrame(
    clf_lr_2.predict_proba(df_test[X_lr_2])[:, 1],
    columns=['failure'],
    index=df_test.index
).to_csv('answer6.csv')